# Notebook to serve as example of how to use polaraveraging
***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import clmm.polaraveraging as pa
import clmm.galaxycluster as gc
import clmm.modeling as modeling
import clmm.plotting as plotting
import clmm

import sys
sys.path.append('./support')
import mock_data as mock

### Define cosmology object

In [ ]:
# if pyccl is installed, use it, else, use astropy FlatLambdaDCM object
# try:
#     import pyccl as ccl
#     cosmo = ccl.Cosmology(Omega_c=0.25, Omega_b = 0.05,h = 0.7,n_s = 0.97,
#                           sigma8 = 0.8, Omega_k = 0.)
# except:
#    from astropy.cosmology import FlatLambdaCDM
#    cosmo = FlatLambdaCDM(H0=70., Om0=0.3, Ob0=0.025)
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70., Om0=0.3, Ob0=0.025)

## 1) Generate cluster object with mock data

#### Preparing config for mock data generation

Mock data will include shape noise, galaxies from redshift distribution and a pdz for each galaxies

In [ ]:
config = {}
config['cluster_id'] = 'mock cluster'
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4
config['ngals'] = 1000
config['Delta'] = 200
config['cosmo'] = cosmo

# noisy_data_z = mock.MockData(config=config)
# noisy_data_z.generate(is_shapenoise=True, is_zdistribution=True,
#                       is_zerr=True)
noisy_data_z = mock.generate_galaxy_catalog(config['cluster_m'],
                                            config['cluster_z'],
                                            config['concentration'],
                                            config['cosmo'],
                                            config['ngals'],
                                            config['Delta'],
                                            'chang13',
                                            shapenoise=0.005,
                                            photoz_sigma_unscaled=0.05)

#### Loading this into a CLMM cluster object

In [ ]:
# At the moment mock data only allow for a cluster centred on (0,0)
cluster_ra = 0.0
cluster_dec = 0.0
gc_object = gc.GalaxyCluster(config['cluster_id'], cluster_ra, cluster_dec,
                             config['cluster_z'], noisy_data_z)
# gc_object.save("GC_from_mock_data.pkl")

## 2) Load cluster object

In [ ]:
# the cluster object contains properties of the lens: ra_l, dec_l,
# z_l and that of the sources : ra_s, dec_s, e1, e1
# cl = gc.load_cluster("GC_from_mock_data.pkl")
cl = gc_object
print("Cluster info = ID:", cl.unique_id, "; ra:", cl.ra,
      "; dec:", cl.dec, "; z_l :", cl.z)
print("The number of source galaxies is :", len(cl.galcat))

In [ ]:
# plot cluster and galaxy positions
plt.scatter(cl.galcat['ra'], cl.galcat['dec'], color='blue', s=1, alpha=0.3)
plt.plot(cl.ra, cl.dec, 'ro')
plt.ylabel('dec', fontsize="large")
plt.xlabel('ra', fontsize="large")

In [ ]:
# check the ellipticities
fig, ax1 = plt.subplots(1, 1)

ax1.scatter(cl.galcat['e1'], cl.galcat['e2'], s=1, alpha=0.2)
ax1.set_xlabel('e1')
ax1.set_ylabel('e2')
ax1.axvline(0, linestyle='dotted', color='black')
ax1.axhline(0, linestyle='dotted', color='black')

## 3) Compute and plot shear profiles

#### Compute angular separation, cross and tangential shear for each galaxies

In [ ]:
# calculate angular separation and tangential shear and cross
# shear for each source galaxy
theta, g_t, g_x = pa.compute_shear(cl, geometry="flat")

#### Plot shear distributions for verification

In [ ]:
# check the shear, now the shear can be accessed through the cluster object
fig, ax1 = plt.subplots(1, 1)

ax1.scatter(cl.galcat['gt'], cl.galcat['gx'], s=1, alpha=0.2)
ax1.set_xlabel('$\\gamma_t$')
ax1.set_ylabel('$\\gamma_x$')
ax1.axvline(0, linestyle='dotted', color='black')
ax1.axhline(0, linestyle='dotted', color='black')

#### Compute transversal and cross shear profiles in units defined by user, using defaults binning 

In [ ]:
profiles = pa.make_shear_profile(cl, "radians", "kpc", cosmo=cosmo)

#### Use function to plot the profiles

In [ ]:
fig, ax = plotting.plot_profiles(cl)

#### Compute transversal and cross shear profiles in units defined by user, using a new binning

In [ ]:
# BE CAREFULL BECAUSE THIS OVERWRITE THE CLUSTER OBJECT
# (better implementation will be done latter)

# define bins
new_bins = pa.make_bins(1, 6, 20)

# compute transversal and cross shear profiles in units
# defined by user, using binning defined by user
new_profiles = pa.make_shear_profile(cl, "radians", "Mpc",
                                     bins=new_bins, cosmo=cosmo)

# plot new profiles
fig1, ax1 = plotting.plot_profiles(cl, "Mpc")

#### You can also access the profiles quantities and play with them

In [ ]:
plt.title('cross shear test')
plt.errorbar(new_profiles['radius'], new_profiles['gx'],
             new_profiles['gx_err'])
plt.title('cross shear test')
plt.axhline(0, linestyle='dotted', color='black')
plt.xlabel("r")
plt.ylabel('$\\gamma_x$')